In [ ]:
#load data from csv
import csv
input_file_path = "/Users/k.abhishek/Documents/customer_clusters/export_all_v2.csv"
def load_data(input_file_path):
    with open(input_file_path, mode ='r') as file:
        csvFile = csv.reader(file)
        data = list(csvFile)
    return data

data = load_data(input_file_path)

In [ ]:
countries_in_focus = ["IN", "CN", "US"]
# countries_in_focus = ["IN"]
country_filter = set(countries_in_focus)

#filtering based on country code
filtered_data = [ d for d in data if d[15] in country_filter]
import random 
random.seed(42)
random.shuffle(filtered_data)

In [ ]:
#find the unique values of legal_cluster_id

legal_cluster_id = set([d[1] for d in filtered_data])
# print(len(legal_cluster_id))

legal_cluster_id_to_data = {}
for d in filtered_data:
    if legal_cluster_id_to_data.get(d[1]) is None:
        legal_cluster_id_to_data[d[1]] = [d]
    else:
        legal_cluster_id_to_data[d[1]].append(d)    
filtered_data = []
num_entries = 100000
test_corpus = []
comparison_corpus = []
count_entries = 0
for legal_cluster_id, entry in legal_cluster_id_to_data.items():
    if count_entries > num_entries:
        break
    if len(entry) > 1:
        filtered_data.extend(entry)
        count_entries += len(entry)
        test_corpus.append(entry[0])
        comparison_corpus.extend(entry[1:])


In [ ]:
def extract_tax_numbers(arr):
    unique_tax_numbers = set()
    for i in range(27, 33):
        if arr[i] and arr[i] != "null":
            unique_tax_numbers.add(arr[i].lower())
    return unique_tax_numbers

def extract_bvd_numbers(arr):
    unique_bvd_numbers = set()
    for i in range(24, 27):
        if arr[i] and arr[i] != "null":
            unique_bvd_numbers.add(arr[i].lower())
    return unique_bvd_numbers

def match_tax(arr, unique_tax_numbers):
    tax = extract_tax_numbers(arr)
    for tax_num in tax:
        if tax_num in unique_tax_numbers:
            return unique_tax_numbers[tax_num]
    return None

def match_bvd(arr, unique_bvd_numbers):
    bvd = extract_bvd_numbers(arr)
    for bvd_num in bvd:
        if bvd_num in unique_bvd_numbers:
            return unique_bvd_numbers[bvd_num]
    return None

In [ ]:
unique_tax_numbers = {}
unique_bvd_numbers = {}
for idx, arr in enumerate(comparison_corpus):
    country_code = arr[15]
    for i in range(24, 27):
        if arr[i] and arr[i] != "null":
            if unique_bvd_numbers.get(arr[i].lower()):
                unique_bvd_numbers[arr[i].lower()].append(arr)
            else:
                unique_bvd_numbers[arr[i].lower()] = [arr]
            # unique_bvd_numbers[arr[i]] = arr
    for i in range(27, 33):
        if arr[i] and arr[i] != "null":
            if unique_tax_numbers.get(arr[i].lower()):
                unique_tax_numbers[arr[i].lower()].append(arr)
            else:
                unique_tax_numbers[arr[i].lower()] = [arr]
            # unique_tax_numbers[arr[i]] = arr


In [56]:
import re
import numpy as np
from rapidfuzz import fuzz
from annoy import AnnoyIndex

import re
stop_words = {'llp', 'ltd', 'llc', 'plc', 'inc', 'corp', 'nv', 'group', 'gmbh', 'sa', 'company', 'corporation', 'limited'}
delimiters = {".", ",", "-", "_", "'", "&", "/", "\\", ":", ";", "(", ")", "[", "]", "{", "}", " "}
def split_company_name(name):
    pattern = '|'.join(map(re.escape, delimiters))
    words = re.split(pattern, name)
    return [word for word in words if word]

def lowercase_company_name(name):
    return name.lower()

# List of common delimiters
def remove_stop_words(words):
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

def preprocess_name_2(trading_name):
    # Lowercase the company name
    trading_name = lowercase_company_name(trading_name)
    # Split the company name into words
    words = split_company_name(trading_name)
    # Remove stop words
    clean_name = remove_stop_words(words)
    return clean_name

# Preprocessing function
def preprocess_name_1(name):
    name = name.lower()
    name = re.sub(r'[^\w\s]', '', name)  # Remove punctuation
    tokens = name.split()
    tokens.sort()  # Sort tokens for token_sort_ratio
    return ' '.join(tokens)

preprocess_name = preprocess_name_2

In [57]:
# Preprocess the entire corpus
# corpus_of_names = ["John Doe", "Jane Smith", "Alice Johnson"]  # Add all million names here
corpus_of_names = [d[4] for d in comparison_corpus]
preprocessed_names = [preprocess_name(name) for name in corpus_of_names]

# Build Annoy index
embedding_dim = 128  # Dimensionality of the embeddings
annoy_index = AnnoyIndex(embedding_dim, 'angular')

# Convert names to vectors (simple bag-of-words vectorization)
def name_to_vector(name):
    vector = np.zeros(embedding_dim)
    tokens = name.split()
    for token in tokens:
        for char in token:
            vector[ord(char) % embedding_dim] += 1  
    return vector

name_vectors = [name_to_vector(name) for name in preprocessed_names]

for i, vector in enumerate(name_vectors):
    # print(i, vector)
    annoy_index.add_item(i, vector)

annoy_index.build(50)  # 10 trees

True

In [58]:
# Fuzzy matching score function
function_selection = {
    "ratio": 1,
    "partial_ratio": 0,
    "token_sort_ratio": 1,
    "token_set_ratio": 1,
    "partial_token_sort_ratio": 0,
    "partial_token_set_ratio": 0
}
def combined_fuzzy_score(name1, name2):
    ratio = fuzz.ratio(name1, name2) * function_selection["ratio"]
    partial_ratio = fuzz.partial_ratio(name1, name2) * function_selection["partial_ratio"]
    token_sort_ratio = fuzz.token_sort_ratio(name1, name2)  * function_selection["token_sort_ratio"]
    token_set_ratio = fuzz.token_set_ratio(name1, name2)    * function_selection["token_set_ratio"]
    partial_token_sort_ratio = fuzz.partial_token_sort_ratio(name1, name2) * function_selection["partial_token_sort_ratio"]
    partial_token_set_ratio = fuzz.partial_token_set_ratio(name1, name2) * function_selection["partial_token_set_ratio"]
    # print(f"ratio: {ratio}, partial_ratio: {partial_ratio}, token_sort_ratio: {token_sort_ratio}, token_set_ratio: {token_set_ratio}, partial_token_sort_ratio: {partial_token_sort_ratio}, partial_token_set_ratio: {partial_token_set_ratio}")
    # print(f"Ratios: {ratio}, {partial_ratio}, {token_sort_ratio}, {token_set_ratio}, {partial_token_sort_ratio}, {partial_token_set_ratio}")
    return sum([ratio, partial_ratio, token_sort_ratio, token_set_ratio, partial_token_sort_ratio, partial_token_set_ratio])/sum(function_selection.values())
    # return max(ratio, partial_ratio, token_sort_ratio, token_set_ratio, partial_token_sort_ratio, partial_token_set_ratio)

In [59]:
# Function to find best match
def find_best_match(input_name, num_candidates=15):
    preprocessed_input = preprocess_name(input_name)
    input_vector = name_to_vector(preprocessed_input)
    
    # Retrieve nearest neighbors using Annoy
    nearest_neighbors = annoy_index.get_nns_by_vector(input_vector, num_candidates, include_distances=False)
    # print(nearest_neighbors)
    best_match_score = 0
    best_match_name = None
    best_match_idx = None
    for neighbor_idx in nearest_neighbors:
        candidate = preprocessed_names[neighbor_idx]
        score = combined_fuzzy_score(preprocessed_input, candidate)
        if score > best_match_score:
            best_match_score = score
            best_match_name = corpus_of_names[neighbor_idx]
            best_match_idx = neighbor_idx
    
    return best_match_name, best_match_score, best_match_idx


In [60]:
from tqdm import tqdm
success = 0
failure = 0
not_applicable = 0
result = {}
result_flagged_existing = {}
qualified_as_match_threshold = 0
correctly_qualified_as_match_threshold = 0
count_entries_name_matching =0
threshold = 85
for arr in tqdm(test_corpus):
    legal_cluster_id = arr[1].lower()
    country_code = arr[15]
    matches = {}
    #tax matching
    tax_match = match_tax(arr, unique_tax_numbers)
    if tax_match:
        success += 1
        matches[1] = tax_match
        result[str(arr)] = matches
        continue
    #bvd matching
    bvd_match = match_bvd(arr, unique_bvd_numbers)
    if bvd_match:
        success += 1
        matches[1] = bvd_match
        result[str(arr)] = matches
        continue
    # print("Name matching")
    company_name = arr[4]
    count_entries_name_matching += 1
    best_match_name, best_match_score, best_match_idx = find_best_match(company_name)
    best_matched_entry = comparison_corpus[best_match_idx]
    if best_match_score > threshold:
        # print(best_match_score)
        qualified_as_match_threshold +=1
        result_flagged_existing[str(arr)] = best_matched_entry
        if legal_cluster_id == best_matched_entry[1]:
            success +=1
            correctly_qualified_as_match_threshold +=1
        else:
            failure +=1
    else:
        failure +=1
    result[str(arr)] = best_matched_entry
    # print(idx+1, success, failure, success + failure)


100%|██████████| 12024/12024 [00:06<00:00, 1939.76it/s]


In [61]:
print(success, failure)
print(qualified_as_match_threshold, correctly_qualified_as_match_threshold)

9853 2171
8506 7775


In [62]:
precision = correctly_qualified_as_match_threshold/qualified_as_match_threshold
print(precision)

0.9140606630613685


In [63]:
import os
save_result_filename = "result_name_matching_threshold_85.json"
save_folder_path = "/Users/k.abhishek/Documents/customer_clusters/experiments/approximate_match_approach/results"
save_result_file_path = os.path.join(save_folder_path, save_result_filename)

def save_result(result, save_result_file_path):
    import json
    with open(save_result_file_path, 'w') as f:
        json.dump(result, f)

save_result(result, save_result_file_path)